In [1]:
# imports
import os
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten, Add, ZeroPadding2D, AveragePooling2D, Activation, BatchNormalization
import tensorflow as tf
from tensorflow.keras import activations
from tensorflow.keras.regularizers import l2

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# preventing tf from utilizing the full GPU
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [6]:
# selecting files from directories
anchor = tf.data.Dataset.list_files(ANC_PATH+'\*.png').take(262)
positive = tf.data.Dataset.list_files(POS_PATH+'\*.png').take(262)
negative = tf.data.Dataset.list_files(NEG_PATH+'\*.png').take(262)

In [ ]:
# dir_test = anchor.as_numpy_iterator()

In [ ]:
# print(dir_test.next())

In [7]:
# preprocessing
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_png(byte_img)    
    # img = tf.image.resize(img, (128,64))
    img = img / 255
    img = img[:,:,0]
    # print(type(img))
    # img = tf.repeat([img], repeats=[3], axis=0)
    # img = tf.transpose(img, perm=[1, 2, 0])
    # print(tf.shape(img))
    return img

In [ ]:
# img = preprocess(r"data\anchor\POAG-000008-2009-02-03-OD.png")

In [ ]:
# img.shape

In [ ]:
# img2 = img[:,:,0]
# img2.shape

In [ ]:
# img.numpy().max()
# plt.imshow(img,cmap="gray")

In [8]:
# creating matching and non-matching pairs
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [ ]:
# data

In [ ]:
# samples = data.as_numpy_iterator()

In [ ]:
# example = samples.next()

In [9]:
# wrapper function for preprecessing 2 images
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [ ]:
# res = preprocess_twin(*example)
# len(res)

In [ ]:
# plt.imshow(res[1])

In [10]:
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [11]:
# setting up data for training
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [12]:
# setting up data for testing
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [ ]:
# embedding = tf.keras.applications.resnet50.ResNet50(
#     input_shape=(128, 64, 3),
#     include_top=True,
#     weights=None
# )

In [13]:
# distancing/differencing layer
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()

    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [14]:
l1 = L1Dist()

In [15]:
siamese_model = tf.keras.models.load_model('siamesemodel_vgg_custom_200+300+200_SGD_1e-2_m0.9_binary.h5', custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [ ]:
# # setting up the siamese architecture
# def make_siamese_model(): 
    
#     input_image = Input(shape=(128,64,3))
    
#     validation_image = Input(shape=(128,64,3))
    
#     siamese_layer = L1Dist()
#     distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
#     classifier = Dense(1, activation='sigmoid')(distances)
    
#     return Model(inputs=[input_image, validation_image], outputs=classifier)

In [ ]:
# siamese_model = make_siamese_model()

In [ ]:
# siamese_model.summary()

In [16]:
# setting up optimizer and loss
binary_cross_loss = tf.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9)

In [17]:
# setting up checkpoints
checkpoint_dir = './vgg-checkpoints/training_checkpoints_SGD_1e-3_200+300+200_vgg_custom'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

In [ ]:
# siamese_model.load_weights("./resnet-checkpoints/training_checkpoints_SGD_1e-2_200_inbuiltresnet_custom")

In [18]:
# custom function for each training step
@tf.function
def train_step(batch):
    with tf.GradientTape() as tape:     
        X = batch[:2]
        y = batch[2]
        
        yhat = siamese_model(X, training=True)
        loss = binary_cross_loss(y, yhat)
    
        
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
    
    return loss

In [19]:
# training and saving checkpoints
def train(data, EPOCHS):
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        for idx, batch in enumerate(data):
            train_step(batch)
            progbar.update(idx+1)
        
        if epoch % 25 == 0: 
            checkpoint.save(file_prefix=checkpoint_prefix)

In [20]:
EPOCHS = 300

In [21]:
train(train_data, EPOCHS)


 Epoch 1/300
23/23 [==============================] - 21s 514ms/step

 Epoch 2/300
23/23 [==============================] - 6s 283ms/step

 Epoch 3/300
23/23 [==============================] - 7s 285ms/step

 Epoch 4/300
23/23 [==============================] - 7s 292ms/step

 Epoch 5/300
23/23 [==============================] - 7s 325ms/step

 Epoch 6/300
23/23 [==============================] - 8s 364ms/step

 Epoch 7/300
23/23 [==============================] - 8s 343ms/step

 Epoch 8/300
23/23 [==============================] - 8s 342ms/step

 Epoch 9/300
23/23 [==============================] - 8s 346ms/step

 Epoch 10/300
23/23 [==============================] - 8s 345ms/step

 Epoch 11/300
23/23 [==============================] - 8s 346ms/step

 Epoch 12/300
23/23 [==============================] - 8s 353ms/step

 Epoch 13/300
23/23 [==============================] - 8s 351ms/step

 Epoch 14/300
23/23 [==============================] - 8s 349ms/step

 Epoch 15/300
23/23 [======

In [23]:
# saving the model
siamese_model.save('siamesemodel_vgg_custom_200+300+200+300_SGD_1e-2_m0.9_binary.h5')

In [ ]:
# from tensorflow.keras.metrics import Precision, Recall

In [ ]:
# test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [ ]:
# y_hat = siamese_model.predict([test_input, test_val])
# y_hat

In [ ]:
# [1 if prediction > 0.5 else 0 for prediction in y_hat ]

In [ ]:
# y_true

In [ ]:
# m = Recall()
# m.update_state(y_true, y_hat)
# m.result().numpy()

In [ ]:
# m = Precision()
# m.update_state(y_true, y_hat)
# m.result().numpy()

In [ ]:
# plt.figure(figsize=(10,8))

# plt.subplot(1,2,1)
# plt.imshow(test_input[0])

# plt.subplot(1,2,2)
# plt.imshow(test_val[0])

# plt.show()